<a href="https://colab.research.google.com/github/UthpalaPitawela/Data_Science_Implementations/blob/main/Text_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import libraries

In [1]:
import pandas as pd
import nltk
import numpy as np

import tensorflow as tf

#libraries for pre processing
from bs4 import BeautifulSoup
!pip install contractions
import contractions
import re
import string
#to remove stop words
nltk.download('stopwords')
from nltk.tokenize import ToktokTokenizer
tokenizer = ToktokTokenizer()
stopword_list = nltk.corpus.stopwords.words('english')
# custom: removing words from list
stopword_list.remove('not')

#for lemmatization
import spacy
nlp = spacy.load('en',parse=True,tag=True, entity=True)

#libraries for feature extraction
import spacy
from spacy import displacy
NER = spacy.load("en_core_web_sm")
from nltk import pos_tag
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

#libraries for evaluation metrics
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

#libraries for SVM
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

#for word embeddings
!git clone https://github.com/facebookresearch/fastText.git
!cd fastText
!pip install fastText
import fasttext.util
fasttext.util.download_model('en', if_exists='ignore')  # English
ft = fasttext.load_model('cc.en.300.bin')



     |████████████████████████████████| 321 kB 7.3 MB/s 
     |████████████████████████████████| 283 kB 53.9 MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85458 sha256=f1edebf9a0c42f140638c880f84ab24acf4841d38db97baef8d36a2eeadef0e6
  Stored in directory: /root/.cache/pip/wheels/25/19/a6/8f363d9939162782bb8439d886469756271abc01f76fbd790f
Successfully built pyahocorasick
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Cloning into 'fastText'...
remote: Enumerating objects: 3854, done.
remote: Total 3854 (delta 0), reused 0 (delta 0), pack-reused 3854
Receiving objects: 100% (3854/3854), 8.22 MiB | 15.86 MiB/s, done.
Resolving deltas: 100% (2417/2417), done.
     |████████████████████████████████| 68 kB 4.4 MB/s 
  Using cached pybind11-2.7.1-py2.py3-none-any.whl (200 kB)
  Created wheel for fastText: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3090426 sha2

Data load from Google Drive

In [2]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

file_id = '1FJYN3RvVIe_zb1XL8w7pbuXRv3S6O38S'
downloaded = drive.CreateFile({'id': file_id})

In [3]:
# Download the file to a local disk as 'dataset.xlsx'.
downloaded.GetContentFile('dataset.xlsx')

In [4]:
# Here it is --
!ls -lha dataset.xlsx

-rw-r--r-- 1 root root 232K Aug 29 16:35 dataset.xlsx


In [5]:
# Now, we can use pandas read_excel after installing the excel importer.
!pip install -q xlrd

import pandas as pd
columns = ['question', 'coarse', 'fine']
df = pd.read_excel('dataset.xlsx', header=None,names=columns)
df

,question,coarse,fine
0,What are the special things we (husband and me...,TTD,TTDSIG
1,What are the companies which organize shark fe...,TTD,TTDOTH
2,Is it safe for female traveller to go alone to...,TGU,TGUHEA
3,What are the best places around Cape Town for ...,TTD,TTDSIG
4,What are the best places to stay for a family ...,ACM,ACMOTH
...,...,...,...
4995,What is the best area to be based for sightsee...,TTD,TTDSIG
4996,What are the good value traditional bars and r...,FOD,FODBAR
4997,What are the hotels near Alicante bus station?,ACM,ACMHOT
4998,Where to stay in La Gomera to mountain biking?,TTD,TTDSPO


Pre processing


*   Remove HTML tags
*   Expand contractions
*   Remove special cases
*   Lowercase all texts
*   Remove stop words
*   Lemmatization









Remove HTML tags

In [6]:
def strip_html_tags(text):
    """remove html tags from text"""
    soup = BeautifulSoup(text, "html.parser")
    stripped_text = soup.get_text(separator=" ")
    return stripped_text


df['html_removed_questions'] = [strip_html_tags(question) for question in df['question']]
df['html_removed_questions'][1]

'What are the companies which organize shark feeding events for scuba divers?'

Expand contractions

In [7]:
def expand_contractions(text):
    """expand shortened words, e.g. don't to do not"""
    text = contractions.fix(text)
    return text

item = "What are the best tourist excursions when it's raining?";

df['contractions_removed_questions'] = [expand_contractions(question) for question in df['html_removed_questions']]
print(item);
print(expand_contractions(item));


What are the best tourist excursions when it's raining?
What are the best tourist excursions when it is raining?


Remove special cases and punctuation

In [8]:
# function to remove special characters
def remove_special_characters_punctuation(text):
    # define the pattern to keep
    pat = r'[^a-zA-z.,!?/:;\"\'\s\n]' 
    formattedText = re.sub(pat, '', text)
    text = ''.join([c for c in formattedText if c not in string.punctuation])
    return text
    
item = "What are the best tourist excursions when it's raining?";

df['special_characters_removed_questions'] = [remove_special_characters_punctuation(question) for question in df['contractions_removed_questions']]
print(item);
print(remove_special_characters_punctuation(item));

What are the best tourist excursions when it's raining?
What are the best tourist excursions when its raining


Lower case text

In [9]:
def lower_case_text(text):
  text = text.strip().lower()
  return text;

item = "Does anyone have the e-mail address for the Ambre resort & spa Mauritius?";

df['lower_cased_questions'] = [lower_case_text(question) for question in df['special_characters_removed_questions']]
print(item);
print(lower_case_text(item));



Does anyone have the e-mail address for the Ambre resort & spa Mauritius?
does anyone have the e-mail address for the ambre resort & spa mauritius?


Remove stop words

In [10]:
# function to remove stopwords
def remove_stopwords(text):
    # convert sentence into token of words
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    # check in lowercase 
    t = [token for token in tokens if token.lower() not in stopword_list]
    text = ' '.join(t)    
    return text

item = "What is the ideal time to visit Mauritius in the year as we are planning to get married in there and spend our honeymoon?";
df['stop_words_removed_questions'] = [remove_stopwords(question) for question in df['lower_cased_questions']]
print(item);
print(remove_stopwords(item));

What is the ideal time to visit Mauritius in the year as we are planning to get married in there and spend our honeymoon?
ideal time visit Mauritius year planning get married spend honeymoon ?


Lemmatization

In [11]:
def get_lem(text):
    text = nlp(text)
    text = ' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in text])
    return text
  
item = "What is the ideal time to visit Mauritius in the year as we are planning to get married in there and spend our honeymoon?";
df['lemmatized_questions'] = [get_lem(question) for question in df['stop_words_removed_questions']]
print(item);
print(get_lem(item));

What is the ideal time to visit Mauritius in the year as we are planning to get married in there and spend our honeymoon?
what be the ideal time to visit Mauritius in the year as we be plan to get marry in there and spend our honeymoon ?


Create Features

*   POS tags
*   Named Entities
*   Head Word Synonyms
*   Bi-grams
*   Head words





















POS Tags

In [12]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
def pos_tagging(text):
    words = nltk.word_tokenize(text)
    lst = [ r[1] for r in pos_tag(words)] 
    return ' '.join(lst)


item = "What is the ideal time to visit Mauritius in the year as we are planning to get married in there and spend our honeymoon?";
df['pos_tagged_questions'] = [pos_tagging(question) for question in df['stop_words_removed_questions']]
print(pos_tagging(item));


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
WP VBZ DT JJ NN TO VB NNP IN DT NN IN PRP VBP VBG TO VB VBN IN EX CC VB PRP$ NN .


Named Entities

In [13]:
def getNamedEntities(text):
  NER_text = NER(text);
  NER_chunk = [];
  for word in NER_text.ents:
    label  = word.label_;
    if label not in NER_chunk:
      NER_chunk.append(word.label_)
  def remove_null(x):
        if '' in x:
            x.remove('')
        return x
  lst = remove_null(NER_chunk)
  # if len(lst)>0:
  #   NER_data = lst[0]
  #   for i in lst[1:]:
  #       NER_data = NER_data + ' ' + i
    # remove_null(NER_data)
  return ' '.join(lst)
  
item = "What is the ideal time to visit Mauritius in the year as we are planning to get married in there and spend our honeymoon?";
df['named_entity_questions'] = [getNamedEntities(question) for question in df['stop_words_removed_questions']]
print(getNamedEntities(item));


GPE DATE


Count vectorizer (BOW)

In [14]:
def get_count_vect(documents):
    vectorizer = CountVectorizer(max_features=1500, min_df=5, max_df=0.7)#, stop_words=stopwords.words('english')
    X = vectorizer.fit_transform(documents).toarray()
    print(X.shape)
    return X

get_count_vect(df['stop_words_removed_questions'])

(5000, 1072)


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

Head word Synonyms

In [15]:
from nltk.corpus import wordnet 
nltk.download('wordnet')
def get_synonyms(words):
    all_synonyms = []
    for word in words.split(' '):
        synonyms = []

        for syn in wordnet.synsets(word):
            for l in syn.lemmas():
                synonyms.append(l.name())

        synonyms = list(set(synonyms))
        all_synonyms += synonyms
        
    return all_synonyms


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [16]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

head_words_synonym_vectorizer = CountVectorizer(tokenizer = get_synonyms,max_features=100,stop_words=stopwords.words('english'))
head_words_synonym_vector = head_words_synonym_vectorizer.fit_transform(df['stop_words_removed_questions'].values).toarray()
print(head_words_synonym_vector)


/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'tween", '1', '1000', '500', 'A', 'AM', 'AN', 'AS', 'Am', 'American_Samoa', 'Artium_Magister', 'As', 'Associate_in_Nursing', 'At', 'Bay_State', 'Be', 'Beaver_State', 'D', 'DO', 'DOE', 'Department_of_Energy', 'Doctor_of_Osteopathy', 'Don', 'Don_River', 'Down', 'Eastern_Samoa', 'Energy', 'Energy_Department', 'Evergreen_State', 'G', 'HA', 'He', 'Hera', 'Here', 'Hoosier_State', 'I', 'IN', 'ISN', 'IT', 'In', 'Indiana', 'International_Relations_and_Security_Network', 'John_L._H._Down', 'K', "KO'd", 'Lapp', 'Lapplander', 'M', 'MA', 'MB', 'ME', 'MT', 'Maine', 'Massachusetts', 'Master_of_Arts', 'MiB', 'More', 'No', 'North_Korean_won', 'O', 'OR', 'Old_Colony', 'Oregon', 'Pine_Tree_State', 'Ra', 'Re', 'S', 'Saame', 'Saami', 'Same', 'Sami', 'Shan', 'Sir_Thomas_More', 'South_Korean_won', 'T', 'Tai_Long',

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


Head Words

In [17]:
# Head word tokenizer
import spacy
nlp = spacy.load("en_core_web_sm")
def head_word_tokenizer(text):
    head_words = []
    for token in nlp(text):
        if token.dep_ == "nsubj" or token.dep_ == "nsubjpass":
            head_words.append(token.text)
#             head_words.append(token.head.text)
    return head_words

In [18]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

head_words_vectorizer = CountVectorizer(tokenizer = head_word_tokenizer,max_features=100,stop_words=stopwords.words('english'))
head_words_vector = head_words_vectorizer.fit_transform(df["stop_words_removed_questions"].values).toarray()
print(head_words_vector)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


Bi-grams

In [19]:
def bigram(text):
    new_words = ""
    pre_word = None
    for word in text.strip().split(' '):
        
        if pre_word is not None:
            new_words += "{}{} ".format(pre_word, word)
        pre_word = word
    return new_words[:-1]

# print(bigram("what are the company which organize shark feeding event for scuba divers "))

item = "What is the ideal time to visit Mauritius in the year as we are planning to get married in there and spend our honeymoon?";
df['bigram_questions'] = [bigram(question) for question in df['stop_words_removed_questions']]

print(bigram(item))

Whatis isthe theideal idealtime timeto tovisit visitMauritius Mauritiusin inthe theyear yearas aswe weare areplanning planningto toget getmarried marriedin inthere thereand andspend spendour ourhoneymoon?


Define course class values

In [20]:
def get_class_values(feature_set='coarse'):
    le = LabelEncoder()
    encoded_classes = le.fit_transform(df[feature_set])
    return encoded_classes

Define evaluation metrics

In [21]:
def PRC_matrics(y_test, prediction):
    # prediction
    precision = precision_score(y_test, prediction, labels=np.unique(prediction), average='micro')*100
    print('Precision: %.3f' % precision)

    # recall
    recall = recall_score(y_test, prediction, labels=np.unique(prediction), average='micro')*100
    print('Recall: %.3f' % recall)
    
    # score
    f1_score = 2 * (precision * recall) / (precision + recall)
    print('F-Measure: %.3f' % f1_score)
    
    
    acc = accuracy_score(y_test, prediction)*100
    print('Accuracy score: %.3f' % acc)
    
    
    cm = confusion_matrix(y_test, prediction)
    print("\nConfustion matrix: \n{}".format(cm))
    
    return precision, recall, f1_score, acc

Define SVM

In [34]:
def train_with_svm(XX, y):
    best_prediction = None
    best_test = None
    best_accuracy = 0
    cv = KFold(n_splits=10, random_state=1, shuffle=True)
    fold = 0
    accuracies = []
    for train_index, test_index in cv.split(XX):
        fold += 1
        X_train, X_test = XX[train_index], XX[test_index]
        y_train, y_test = y[train_index], y[test_index]

        SVM = SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
        SVM.fit(X_train,y_train)
        predictions_SVM1 = SVM.predict(X_test)
        acc = accuracy_score(predictions_SVM1, y_test)*100
        if best_accuracy < acc:
            best_accuracy = acc
            best_prediction = predictions_SVM1
            best_test = y_test
            best_model = SVM
        accuracies.append(acc)
        print("Fold - {} - {} - {:.2f}".format(fold, "Accuracy-> ",acc))

    print("Mean Accuracy {:.2f} \nStd Accuracy {:.2f}\n\n".format(np.mean(accuracies), np.std(accuracies)))
    
    print("Best accuracy : {}".format(best_accuracy))
    PRC_matrics(best_test, best_prediction)

**Question 01: A traditional ML classifier s.a. SVM or Logistic Regression with at least 5  of the features mentioned in the paper**

Selected Features

*   POS tags
*   Named Entities
*   Head words
*   Head Word Synonyms
*   Bi-grams

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

from scipy.sparse import coo_matrix, csr_matrix, hstack


coarse = get_class_values('coarse')

X_lem = df['lemmatized_questions']
tfidf_lem = TfidfVectorizer(max_features=5000)
tfidf_lem.fit(X_lem)

X_pos = df['pos_tagged_questions']

X_ne = df['named_entity_questions']

X_bigram = df['bigram_questions']


XX = csr_matrix(hstack([tfidf_lem.transform(X_lem) ,get_count_vect(X_pos), get_count_vect(X_ne), get_count_vect(X_bigram),
                        head_words_vector,head_words_synonym_vector]))
XX.shape

(5000, 22)
(5000, 13)
(5000, 344)


(5000, 5561)

Feature Selection

1. Select k best features

In [24]:

from sklearn.feature_selection import SelectKBest, chi2

X_new = SelectKBest(chi2, k=3500).fit_transform(XX, coarse)
X_new.shape

(5000, 3500)

2. Remove features considering a specified threshold

In [25]:
from sklearn.feature_selection import VarianceThreshold
X_new_var = VarianceThreshold(threshold=(0.01)).fit_transform(XX)
X_new_var.shape

(5000, 132)

Training using the defined SVM

1. Train with coarse classes

In [35]:
train_with_svm(XX, get_class_values('coarse'))

Fold - 1 - Accuracy->  - 81.60
Fold - 2 - Accuracy->  - 78.60
Fold - 3 - Accuracy->  - 76.00
Fold - 4 - Accuracy->  - 77.80
Fold - 5 - Accuracy->  - 79.40
Fold - 6 - Accuracy->  - 78.40
Fold - 7 - Accuracy->  - 76.00
Fold - 8 - Accuracy->  - 77.00
Fold - 9 - Accuracy->  - 78.80
Fold - 10 - Accuracy->  - 78.40
Mean Accuracy 78.20 
Std Accuracy 1.58


Best accuracy : 81.6
Precision: 81.600
Recall: 81.600
F-Measure: 81.600
Accuracy score: 81.600

Confustion matrix: 
[[ 69   1   3   2   2   4   0]
 [  0  14   2   0   2   7   0]
 [  4   2  41   5   0   4   0]
 [  4   0   1  86   6   9   0]
 [  2   0   2   4  85   3   0]
 [  5   3   0   7   3 100   0]
 [  0   0   0   2   0   3  13]]


2. Train with fine classes

In [27]:
train_with_svm(XX, get_class_values('fine'))

Fold - 1 - SVM Accuracy Score ->  - 52.60
Fold - 2 - SVM Accuracy Score ->  - 51.40
Fold - 3 - SVM Accuracy Score ->  - 49.40
Fold - 4 - SVM Accuracy Score ->  - 52.60
Fold - 5 - SVM Accuracy Score ->  - 52.20
Fold - 6 - SVM Accuracy Score ->  - 51.20
Fold - 7 - SVM Accuracy Score ->  - 50.40
Fold - 8 - SVM Accuracy Score ->  - 51.20
Fold - 9 - SVM Accuracy Score ->  - 53.40
Fold - 10 - SVM Accuracy Score ->  - 52.00
Mean Accuracy 51.64 
Std Accuracy 1.11


Best accuracy : 53.400000000000006
Precision: 53.400
Recall: 55.394
F-Measure: 54.379
Accuracy score: 53.400

Confustion matrix: 
[[0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 7 0 0]
 [0 0 0 ... 1 2 0]
 [0 0 0 ... 1 0 1]]


**Question 02: 2. A traditional ML classifier s.a. SVM or Logistic Regression with word embedding features s.a. word2vec or fasttext.**


Word Embeddings: Using fasttext

In [30]:
def get_embedding(sentence):    
    embedding=ft.get_sentence_vector(sentence)
    return embedding

In [31]:
X_embedding = np.array([get_embedding(question) for question in df['stop_words_removed_questions'].values])
le = LabelEncoder()

Train with SVM for coarse classes

In [32]:
train_with_svm(X_embedding, get_class_values('coarse'))

Fold - 1 - SVM Accuracy Score ->  - 79.60
Fold - 2 - SVM Accuracy Score ->  - 79.20
Fold - 3 - SVM Accuracy Score ->  - 73.20
Fold - 4 - SVM Accuracy Score ->  - 78.00
Fold - 5 - SVM Accuracy Score ->  - 79.00
Fold - 6 - SVM Accuracy Score ->  - 79.00
Fold - 7 - SVM Accuracy Score ->  - 76.60
Fold - 8 - SVM Accuracy Score ->  - 78.20
Fold - 9 - SVM Accuracy Score ->  - 80.20
Fold - 10 - SVM Accuracy Score ->  - 80.20
Mean Accuracy 78.32 
Std Accuracy 1.99


Best accuracy : 80.2
Precision: 80.200
Recall: 80.522
F-Measure: 80.361
Accuracy score: 80.200

Confustion matrix: 
[[  0   0   0   0   0   0   0   1   0]
 [  0  54   0   2   5   0   2   7   1]
 [  0   1  10   1   0   0   0   5   0]
 [  0   2   2  46   1   0   1   2   0]
 [  0   3   0   3  98   0   4  18   0]
 [  0   0   0   0   1   0   0   0   0]
 [  0   1   1   0   6   0  80   7   0]
 [  0   4   2   1   9   0   3 101   1]
 [  0   0   0   0   1   0   0   1  12]]


Train with SVM for fine classes

In [33]:
train_with_svm(X_embedding, get_class_values('fine'))

Fold - 1 - SVM Accuracy Score ->  - 54.40
Fold - 2 - SVM Accuracy Score ->  - 56.00
Fold - 3 - SVM Accuracy Score ->  - 54.80
Fold - 4 - SVM Accuracy Score ->  - 52.00
Fold - 5 - SVM Accuracy Score ->  - 53.40
Fold - 6 - SVM Accuracy Score ->  - 55.40
Fold - 7 - SVM Accuracy Score ->  - 54.20
Fold - 8 - SVM Accuracy Score ->  - 54.20
Fold - 9 - SVM Accuracy Score ->  - 57.00
Fold - 10 - SVM Accuracy Score ->  - 53.40
Mean Accuracy 54.48 
Std Accuracy 1.35


Best accuracy : 56.99999999999999
Precision: 57.000
Recall: 67.059
F-Measure: 61.622
Accuracy score: 57.000

Confustion matrix: 
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 6 0 0]
 [0 0 0 ... 3 0 0]
 [0 0 0 ... 2 0 0]]


**Question 03: A NN classifier s.a. an LSTM for classification**